# Load FAISS Index in Google Colab

This notebook demonstrates how to load and use the FAISS index exported from the Chrome extension backend.


In [ ]:
# Install required packages
%pip install faiss-cpu sentence-transformers numpy


In [ ]:
import faiss
import numpy as np
import pickle
import zipfile
from sentence_transformers import SentenceTransformer
from google.colab import files


In [ ]:
# Upload the exported zip file
uploaded = files.upload()
zip_filename = list(uploaded.keys())[0]


In [ ]:
# Extract the zip file
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('faiss_index')


In [ ]:
# Load FAISS index and metadata
index = faiss.read_index('faiss_index/index.faiss')
with open('faiss_index/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

print(f"Index loaded: {index.ntotal} vectors")
print(f"Metadata: {len(metadata)} entries")


In [ ]:
# Load the embedding model (same as used for indexing)
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
# Search function
def search(query, top_k=5):
    # Encode query
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).astype('float32')
    
    # Search
    k = min(top_k, index.ntotal)
    distances, indices = index.search(query_embedding, k)
    
    # Get results
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(metadata):
            result = metadata[idx].copy()
            result['distance'] = float(distances[0][i])
            results.append(result)
    
    return results


In [ ]:
# Example search
query = "your search query here"
results = search(query, top_k=5)

for i, result in enumerate(results, 1):
    print(f"\nResult {i}:")
    print(f"Title: {result['title']}")
    print(f"URL: {result['url']}")
    print(f"Distance: {result['distance']:.4f}")
    print(f"Chunk: {result['chunk_text'][:200]}...")
